In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,34574
2,Huelva,Confirmados PDIA 14 días,1103
3,Huelva,Tasa PDIA 14 días,"214,93851939902956"
4,Huelva,Confirmados PDIA 7 días,644
5,Huelva,Total Confirmados,34761
6,Huelva,Curados,32509
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  34574.0
/tmp/ipykernel_114377/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9447.0
/tmp/ipykernel_114377/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_114377/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_114377/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_114377/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 841 personas en los últimos 7 días 

Un positivo PCR cada 512 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,34574.0,644.0,1103.0,513170.0,125.494476,214.938519,275.0
Huelva-Costa,20043.0,455.0,746.0,289548.0,157.141476,257.642947,183.0
Huelva (capital),9447.0,171.0,281.0,143837.0,118.884571,195.360026,67.0
Condado-Campiña,11163.0,150.0,286.0,156231.0,96.011675,183.062260,65.0
Isla Cristina,2361.0,104.0,164.0,21393.0,486.140326,766.605899,32.0
Lepe,2277.0,67.0,95.0,27880.0,240.315638,340.746055,31.0
Ayamonte,1431.0,43.0,68.0,21104.0,203.752843,322.213798,20.0
Aljaraque,1226.0,33.0,45.0,21474.0,153.674211,209.555742,18.0
Niebla,296.0,32.0,47.0,4116.0,777.453839,1141.885326,15.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Santa Bárbara de Casa,35.0,6.0,19.0,1043.0,575.263663,1821.668265,0.0
Niebla,296.0,32.0,47.0,4116.0,777.453839,1141.885326,15.0
Isla Cristina,2361.0,104.0,164.0,21393.0,486.140326,766.605899,32.0
Alosno,207.0,6.0,19.0,3933.0,152.555301,483.091787,0.0
Zarza-Perrunal (La),33.0,3.0,6.0,1253.0,239.425379,478.850758,2.0
Lucena del Puerto,279.0,4.0,12.0,3261.0,122.661760,367.985281,1.0
Puerto Moral,5.0,1.0,1.0,278.0,359.712230,359.712230,0.0
Lepe,2277.0,67.0,95.0,27880.0,240.315638,340.746055,31.0
Ayamonte,1431.0,43.0,68.0,21104.0,203.752843,322.213798,20.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,680.0,2.0,15.0,12750.0,15.686275,117.647059,2.0,0.133333
Bollullos Par del Condado,1059.0,6.0,21.0,14387.0,41.704316,145.965107,0.0,0.285714
Santa Bárbara de Casa,35.0,6.0,19.0,1043.0,575.263663,1821.668265,0.0,0.315789
Alosno,207.0,6.0,19.0,3933.0,152.555301,483.091787,0.0,0.315789
Nerva,183.0,1.0,3.0,5169.0,19.346102,58.038305,1.0,0.333333
Beas,235.0,1.0,3.0,4341.0,23.036167,69.108500,1.0,0.333333
Lucena del Puerto,279.0,4.0,12.0,3261.0,122.661760,367.985281,1.0,0.333333
Bonares,318.0,5.0,14.0,6060.0,82.508251,231.023102,2.0,0.357143
Almonte,1954.0,13.0,35.0,24507.0,53.046068,142.816338,4.0,0.371429
